# GLiNER PII Detection Example

This notebook shows how to use GLiNER for PII detection and PII masking in NeMo Guardrails.

GLiNER is a Generalist and Lightweight Model for Named Entity Recognition that can detect a wide range of entity types, including comprehensive PII categories.


## Prerequisites

Before running this notebook, ensure you have the GLiNER server running:

```bash
# Install dependencies
pip install gliner torch fastapi uvicorn

# Start the example server (uses nvidia/gliner-PII model by default)
python examples/deployment/gliner_server/gliner_server.py --host 0.0.0.0 --port 1235
```

For more details, see the [GLiNER server deployment guide](../deployment/gliner_server/README.md).


## PII Detection


### Import libraries


In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from nemoguardrails import LLMRails, RailsConfig

### Create rails with GLiNER PII detection

For this step you'll need your OpenAI API key and the GLiNER server running.

For more details on GLiNER integration, check out this [user guide](../../docs/user-guides/community/gliner.md).


In [ ]:
# Make sure OPENAI_API_KEY is set in your environment
# os.environ["OPENAI_API_KEY"] = "YOUR OPENAI API KEY"

YAML_CONFIG = """
models:
  - type: main
    engine: openai
    model: gpt-4o-mini

rails:
  config:
    gliner:
      server_endpoint: http://localhost:1235/v1/extract
      threshold: 0.5
      input:
        entities:
          - first_name
          - last_name
          - street_address
          - email
      output:
        entities:
          - first_name
          - last_name
          - street_address
          - email
  input:
    flows:
      - gliner detect pii on input

  output:
    flows:
      - gliner detect pii on output
"""


config = RailsConfig.from_content(yaml_content=YAML_CONFIG)
rails = LLMRails(config)

### Input rails

When PII is detected in user input, the request is blocked.


In [ ]:
response = rails.generate(
    messages=[{"role": "user", "content": "Hello! I'm John. My email id is text@gmail.com. I live in California, USA."}]
)

info = rails.explain()

print("Response")
print("----------------------------------------")
print(response["content"])


print("\n\nColang history")
print("----------------------------------------")
print(info.colang_history)

print("\n\nLLM calls summary")
print("----------------------------------------")
info.print_llm_calls_summary()

### Output rails

When PII is detected in the LLM output, the response is blocked.


In [ ]:
response = rails.generate(messages=[{"role": "user", "content": "give me a sample email id"}])

info = rails.explain()

print("Response")
print("----------------------------------------\n\n")
print(response["content"])


print("\n\nColang history")
print("----------------------------------------")
print(info.colang_history)

print("\n\nLLM calls summary")
print("----------------------------------------")
info.print_llm_calls_summary()


print("\n\nCompletions where PII was detected!")
print("----------------------------------------")
print(info.llm_calls[0].completion)

## PII Masking

Instead of blocking messages with PII, you can mask the PII before processing.


### Input rails


In [ ]:
YAML_CONFIG = """
models:
  - type: main
    engine: openai
    model: gpt-4o-mini

rails:
  config:
    gliner:
      server_endpoint: http://localhost:1235/v1/extract
      threshold: 0.5
      input:
        entities:
          - first_name
          - city
          - country
          - email
  input:
    flows:
      - gliner mask pii on input
"""


config = RailsConfig.from_content(yaml_content=YAML_CONFIG)
rails = LLMRails(config)

In [ ]:
response = rails.generate(
    messages=[{"role": "user", "content": "Hello! I'm John. My email id is text@gmail.com. I live in California, USA."}]
)

info = rails.explain()

print("Response")
print("----------------------------------------")
print(response["content"])


print("\n\nColang history")
print("----------------------------------------")
print(info.colang_history)

print("\n\nLLM calls summary")
print("----------------------------------------")
info.print_llm_calls_summary()

### Output rails


In [ ]:
YAML_CONFIG = """
models:
  - type: main
    engine: openai
    model: gpt-4o-mini

rails:
  config:
    gliner:
      server_endpoint: http://localhost:1235/v1/extract
      threshold: 0.5
      output:
        entities:
          - first_name
          - city
          - country
          - email
  output:
    flows:
      - gliner mask pii on output
"""


config = RailsConfig.from_content(yaml_content=YAML_CONFIG)
rails = LLMRails(config)

In [ ]:
response = rails.generate(messages=[{"role": "user", "content": "give me a sample email id"}])

info = rails.explain()

print("Response")
print("----------------------------------------\n\n")
print(response["content"])


print("\n\nColang history")
print("----------------------------------------")
print(info.colang_history)

print("\n\nLLM calls summary")
print("----------------------------------------")
info.print_llm_calls_summary()


print("\n\nOriginal completion (before masking)")
print("----------------------------------------")
print(info.llm_calls[0].completion)

## Supported Entity Types

The GLiNER server (using the `nvidia/gliner-PII` model) supports these PII categories:

| Category | Entity Types |
|----------|-------------|
| Personal Identifiers | `first_name`, `last_name`, `ssn`, `date_of_birth`, `age`, `gender` |
| Contact Information | `email`, `phone_number`, `fax_number`, `street_address`, `city`, `state`, `postcode`, `country`, `county` |
| Financial | `credit_debit_card`, `cvv`, `bank_routing_number`, `account_number`, `swift_bic`, `tax_id` |
| Technical | `ipv4`, `ipv6`, `mac_address`, `url`, `api_key`, `password`, `pin`, `http_cookie` |
| Identification | `national_id`, `license_plate`, `vehicle_identifier`, `employee_id`, `customer_id`, `unique_id` |
| Sensitive Attributes | `sexuality`, `political_view`, `race_ethnicity`, `religious_belief`, `blood_type` |
